In [ ]:
# Add the current user to the 'docker' group (Linux).
# This allows running Docker commands without prefixing them with 'sudo'.
# Note: you must log out and log back in (or restart the shell) for the group change to take effect.
sudo usermod -aG docker $USER

In [ ]:
# Build the Docker image locally and tag it as 'iris-api'.
# -t sets the image name (and optional tag). The final '.' is the build context (current directory).
docker build -t iris-api .

# Run the Docker container and map container port 8200 to host port 8200.
# Use '-d' to run in detached mode, or add '-it' for interactive shells if needed.
docker run -p 8200:8200 iris-api


In [ ]:
# Create a Google Cloud Artifact Registry repository for Docker images.
# Replace <PROJECT> and <LOCATION> as needed (example uses 'us-central1' and the project from the repo).
gcloud artifacts repositories create iris-repo \
  --repository-format=docker \
  --location=us-central1 \
  --description="Docker repo for IRIS API"

In [ ]:
# Configure Docker credential helper so 'docker push' can authenticate with Artifact Registry.
# This updates your local Docker config to use gcloud credentials for the specified registry host.
gcloud auth configure-docker us-central1-docker.pkg.dev

In [ ]:
# Tag the locally built image with the Artifact Registry repository path.
# Format: <LOCATION>-docker.pkg.dev/<PROJECT>/<REPO>/<IMAGE>:<TAG>
docker tag iris-api us-central1-docker.pkg.dev/buoyant-country-473106-a3/iris-repo/iris-api:latest

In [ ]:
# Push the tagged image to Artifact Registry so it can be deployed from the cloud.
docker push us-central1-docker.pkg.dev/buoyant-country-473106-a3/iris-repo/iris-api:latest

In [ ]:
# Create a Google Kubernetes Engine (GKE) cluster using Autopilot mode.
# Autopilot manages node provisioning and autoscaling for you.
gcloud container clusters create-auto iris-cluster \
    --location=us-central1 \
    --project=buoyant-country-473106-a3

In [ ]:
# Install the GKE auth plugin so kubectl can authenticate to GKE using gcloud.
gcloud components install gke-gcloud-auth-plugin
# Alternative (Debian/Ubuntu):
# sudo apt-get install google-cloud-cli-gke-gcloud-auth-plugin

In [ ]:
# Fetch cluster credentials and configure kubectl to talk to the GKE cluster.
# This sets the current context to the specified cluster so subsequent kubectl commands target it.
gcloud container clusters get-credentials iris-cluster --zone us-central1 --project buoyant-country-473106-a3

In [ ]:
# Update the image for the 'iris-api' deployment to a new image tag and watch the rollout.
# Ensure the image path and tag match what was pushed to Artifact Registry.
kubectl set image deployment/iris-api iris-api=us-central1-docker.pkg.dev/buoyant-country-473106-a3/iris-repo/iris-api:latest-new
kubectl rollout status deployment/iris-api

In [ ]:
# Common kubectl commands for applying manifests, inspecting resources, and cleaning up.
# Apply deployment manifest to create/update resources from k8s/deployment.yaml
kubectl apply -f k8s/deployment.yaml 
# Apply service manifest to expose the deployment
kubectl apply -f k8s/service.yaml

# List pods in the current namespace
kubectl get pods
# List services in the current namespace
kubectl get services

# Delete the deployment and service when you no longer need them
kubectl delete deployment iris-api 
kubectl delete service iris-api-service